# Experiment Result Analysis


In [8]:
import json
from defines import EVAL_PATH

In [9]:
method_name = "LocSearchAdv"
eval_dir = f"{EVAL_PATH}/{method_name}"

In [10]:
success = []
failed = []
total_exp = 0
with open(f'{eval_dir}/{method_name}_exp.json') as f:
    results = json.load(f)
    total_exp = len(results)
    for res in results:
        true_label = res["true_label_idx"]
        perturbed_label = res["perturbed_topk_indices"][0]
        if true_label == perturbed_label:
            failed.append(res)
        else:
            success.append(res)

In [11]:
num_success = len(success)
success_rate = num_success / total_exp

total = 0.0
for exp in success:
    perturbed_prob = exp["perturbed_topk_probabilities"][0]
    total += perturbed_prob

print("Success Rate: {}\nConf: {}".format(success_rate, total/num_success))

Success Rate: 0.56
Conf: 0.3153872308986528
